#### 0.Data

In [17]:
import pandas as pd
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
coupon = pd.read_excel("http://byungwan.com/class/Coupon_Assignment2.xlsx")
coupon.head()

,CID,Gender,Age,CClass,Discount,Visits,Email,SMS,MStatus,Purchases,Sales
0,1,Male,25,2,Coupon Not Used,1,Opt-in,Opt-in,Married,1,0.5050
1,2,Female,17,2,Coupon Not Used,1,Opt-in,Opt-in,Married,1,0.3300
2,3,Female,38,2,Coupon Not Used,3,Opt-in,Opt-in,Married,1,0.2550
3,4,Female,36,2,Coupon Not Used,6,Opt-in,Opt-in,Single,3,1.8762
4,5,Female,37,2,Coupon Not Used,7,Opt-in,Opt-in,Single,5,6.0940


In [3]:
coupon = pd.get_dummies(coupon, columns=['Gender','Discount','Email','SMS','MStatus'], drop_first=True, dtype=int)

In [4]:
coupon.head()

,CID,Age,CClass,Visits,Purchases,Sales,Gender_Male,Discount_Coupon Used,Email_Opt-out,SMS_Opt-out,MStatus_Single
0,1,25,2,1,1,0.5050,1,0,0,0,0
1,2,17,2,1,1,0.3300,0,0,0,0,0
2,3,38,2,3,1,0.2550,0,0,0,0,0
3,4,36,2,6,3,1.8762,0,0,0,0,1
4,5,37,2,7,5,6.0940,0,0,0,0,1


In [5]:
coupon['Discount_Coupon Used'].value_counts()

Discount_Coupon Used
0    927
1     73
Name: count, dtype: int64

#### 1. Balance Data (Downsampling)

In [6]:
coupon0 = coupon[coupon['Discount_Coupon Used']==0]
coupon1 = coupon[coupon['Discount_Coupon Used']==1]

In [8]:
coupon0_downsampled = resample(coupon0, replace=False, n_samples=73, random_state=0)

In [11]:
coupon_downsampled = pd.concat([coupon0_downsampled, coupon1], axis=0)
coupon_downsampled['Discount_Coupon Used'].value_counts()

Discount_Coupon Used
0    73
1    73
Name: count, dtype: int64

In [13]:
coupon_downsampled.head()

,CID,Age,CClass,Visits,Purchases,Sales,Gender_Male,Discount_Coupon Used,Email_Opt-out,SMS_Opt-out,MStatus_Single
339,340,24,2,50,11,5.0964,0,0,0,0,0
386,387,40,2,13,9,6.5546,0,0,0,0,0
730,731,31,2,2,1,0.4000,1,0,0,0,0
984,985,24,1,2,0,0.0000,1,0,0,0,0
886,887,25,1,0,0,0.0000,0,0,0,0,0


In [14]:
coupon_downsampled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146 entries, 339 to 970
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   CID                   146 non-null    int64  
 1   Age                   146 non-null    int64  
 2   CClass                146 non-null    int64  
 3   Visits                146 non-null    int64  
 4   Purchases             146 non-null    int64  
 5   Sales                 146 non-null    float64
 6   Gender_Male           146 non-null    int64  
 7   Discount_Coupon Used  146 non-null    int64  
 8   Email_Opt-out         146 non-null    int64  
 9   SMS_Opt-out           146 non-null    int64  
 10  MStatus_Single        146 non-null    int64  
dtypes: float64(1), int64(10)
memory usage: 13.7 KB


#### 2. Preprocessing

In [15]:
y = coupon_downsampled.iloc[:, 7]
x = coupon_downsampled.iloc[:, [1,2,3,4,5,6,8,9,10]]

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

#### 3. Scaling

In [ ]:
scaler = StandardScaler()
scaler_model = scaler.fit(x)
